In [ ]:
%run SequencingSimulator.ipynb

In [ ]:
#test for getSigma function

assert getSigma(35) == 1
assert getSigma(125) == 1
assert getSigma(45) == 2
assert getSigma(115) == 2
assert getSigma(65) == 5
assert getSigma(95) == 5
assert getSigma(75) == 10

In [ ]:
#test for generateQuality function

def calculateSum(readFile, readLength, quality, fileContent):
    sum = 0
    for ch in fileContent:
        sum += ord(ch)
    sum = (int)(sum/readLength)
    return sum

def calculateLengthOfFileContent(readFile):
    fileContent = readFile.read().rstrip('\n')
    return fileContent
    
def generateQualityTest(quality, readLength, readFile):
    generateQuality(quality, readLength, readFile)
    readFile.flush()
    readFile.seek(0)
    fileContent = calculateLengthOfFileContent(readFile)
    assert len(fileContent) == readLength
    sum = calculateSum(readFile, readLength, quality, fileContent)
    assert sum <= quality + 2
    assert sum >= quality - 2
    
readFile = open("Read1.txt","w+")

readLength = 100
quality = 70
generateQualityTest(quality, readLength, readFile)

readFile.seek(0)

readLength = 150
quality = 50
generateQualityTest(quality, readLength, readFile)

In [ ]:
#test for normalDistributionNumbersGenerator

In [ ]:
#test for numOfFragments function

In [ ]:
#test for generateReads function

In [ ]:
#test for insertMutations function

In [ ]:
#test for createPairEndPairs function